In [32]:
import re

def updatefile(fname):
    f = open(fname + '.xml')
    outf = open(fname + '-work.xml','w')
    inbody = 0
    prevpage = 0
    prevvol = 0
    for l in f:
        if(re.search('<body>',l)):
            inbody = 1
        if(not inbody):
            print(l,end='',file=outf)
            continue
            
        if( re.search('<(note|fig)',l)):
            print(l,end='',file=outf)
            continue
        
        workl = l
        while(re.search('<pb n="v([0-9][0-9])p([0-9][0-9][0-9])',workl)):
            m = re.search('<pb n="v([0-9][0-9])p([0-9][0-9][0-9])',workl)
            if(m):
                curvol = int(m[1])
                curpage = int(m[2])
                if(not curvol == prevvol):
                    if(curvol == prevvol + 1):
                        prevpage = 0
                    elif(  not curvol == prevvol + 11):
                        print('jump vols',prevvol,curvol,l)
                if( not curpage == prevpage + 1):
                    print('jumppage',curvol,prevpage,curpage)
                prevvol = curvol
                prevpage = curpage
            workl = re.sub('<pb n="v([0-9][0-9])p([0-9][0-9][0-9])',' ',workl,1)
        l = re.sub('\\b([IVXL0-9]+\.|St\.|Dr\.|Mr\.|[A-Z]\.)\s+','\g<1>---',l)

        #l = re.sub('\s*(<pb[^>]+>|<p [^>]+>)','\n\g<1>\n',l)
        l = re.sub('([\.!?]|[\.!?]</quote>|[\.!?]</hi>)(\s+|<ref[^>]+>\s+|\s*<pb[^>]+>\s*)([A-Z]|<quote)','\g<1>\g<2>\n\n\g<3>',l)
        l = re.sub('(<p [^>]+>)','\g<1>\n',l)
        l = re.sub('(</p>)','\n\g<1>',l)
        l = re.sub('---',' ',l)
        print(l,end='',file=outf)

    f.close()
    outf.close()
    
    outf = open(fname + '-out.xml','w')
    f = open(fname + '-work.xml')

    inbody = 0
    curpage = ''
    for l in f:
        if(re.search('<body>',l)):
            inbody = 1
        if(not inbody):
            print(l,end='',file=outf)
            continue
        l = re.sub('(id="ch)([0-9][0-9])\.','\g<1>\g<2>',l)
        l = re.sub('(id="ch)([0-9])\.','\g<1>0\g<2>',l)
        l = re.sub('(pa)([0-9])"','\g<1>0\g<2>"',l)
        m = re.search('<p xml:id="(ch[0-9]+pa[0-9]+)"',l)
        if(m):
            curparid = m[1]
            sentnum = 1
            
        l = re.sub('(<p [^>]+pa)([0-9]+)"','\g<1>\g<2>" n="\g<2>"',l)
        l = re.sub(' n="[0]*',' n="',l)
        if(re.search('^<(div|list|head|fig|note|item|l\\b|date)',l)):
            print(l,end='',file=outf)
            continue
        workl = re.sub('<[^>]+>','',l)
        if(workl[0].isupper()):
            sentnums = str(sentnum)
            sentnums = re.sub('^([0-9]$)','00\g<1>',sentnums)
            sentnums = re.sub('^([0-9][0-9]$)','0\g<1>',sentnums)
            sentid = curparid + 'se' + sentnums
            print('<s xml:id="'+sentid+'" n="' + str(sentnum)+'"/>',file=outf,end='')
            sentnum = sentnum + 1
        
        print(l,file=outf,end='')
        
    f.close()
    outf.close()

updatefile('bury-all')
print('done')

done


In [5]:
import re


stopwords = {}
stopwords['A'] = 1
stopwords['By'] = 1
stopwords['I'] = 1
stopwords['Yet'] = 1
stopwords['X'] = 1

def check2lower(curword):        
    if(curword in stopwords):
        return(curword.lower())
           
    if(curword.lower() in wordclasses and wordclasses[curword.lower()]> wordclasses[curword]):
        return(curword.lower())

    return('')

f = open('bury-all-out.xml')

wordclasses = {}
for l in f:
    if( not re.search('^<s ',l)):
        continue
    l = re.sub('<[^>]+>',' ',l)
    l = re.sub('[;:,!?\']',' ',l)
    for word in l.split():
        if( word in wordclasses):
            wordclasses[word] = wordclasses[word] + 1
        else:
            wordclasses[word] = 1

f.close()

f = open('bury-all-out.xml')
outf = open('/tmp/bury-page.xml','w')
for l in f:
    l = re.sub('(<pb )','\n\g<1>',l)
    print(l,end='',file=outf)
outf.close()

f.close()
connectors = {}
connectors['of'] = 1
connectors['the'] = 1
curpage = ''
f = open('/tmp/bury-page.xml')

prevword = ''
prevword2 = ''

unigrams = {}
bigrams = {}
trigrams = {}

curid = ''
for l in f:
    m = re.search('<s xml:id="(ch[0-9]+[^"]+)"',l)
    if(m):
        curid = m[1]

    if(not re.search('^ch48',curid)):
        continue
    
    if(not re.search('(<s |<pb )',l)):
        continue

    m = re.search('<pb n="(v[0-9][0-9]p[0-9][0-9][0-9])"',l)
    if(m):
        curpage = m[1]
        #print(curpage)
    

    m = re.search('(<quote>|<s [^>]+>|<hi rend="small-caps">)([A-Z][a-z\-]*)',l)
    if(m):
        curword = m[2]
        subw = check2lower(curword)
        if(subw):
            l = re.sub(curword +'\\b',subw,l)

    workl = re.sub('<[^>]+>',' ',l)
    workl = re.sub('(St\.)\s+([A-Z])','\g<1>_\g<2>',workl)

    workl = re.sub('([;:,\.!?\'][\s\n])',' \g<1>',workl)
    workl = re.sub('([\']s[\s\n,;:])',' \g<1>',workl)
    
    workl = re.sub('\\bThe\\b','the',workl)

    m = re.search('^\s*([A-Z][a-z\-]+)',workl)
    if(m):
        curword = m[1]
        subw = check2lower(curword)
        if(subw):
            workl = re.sub(curword +'\\b',subw,workl)

    print('\n',curid,'\t',workl,sep='',end='')
    for word in workl.split():
        if(word[0].isupper() and not word in stopwords):
            if(word in unigrams):
                unigrams[word] = unigrams[word] +1
            else:
                unigrams[word] = 1
            print(curpage,word,unigrams[word])
            if( prevword and prevword[0].isupper()):
                curbigram = prevword + ' ' + word
                if(curbigram in bigrams):
                    bigrams[curbigram] = bigrams[curbigram] +1
                else:
                    bigrams[curbigram] = 1
                print(curpage,curbigram,bigrams[curbigram])
        
            if( prevword and prevword[0].isupper() and prevword2 and prevword2[0].isupper()):
                curtrigram = prevword2 + ' ' + prevword + ' ' + word
                if(curtrigram in trigrams):
                    trigrams[curtrigram] = trigrams[curtrigram] +1
                else:
                    trigrams[curtrigram] = 1
                print(curpage,curtrigram,trigrams[curtrigram])
        
            if(prevword in connectors and prevword2 and prevword2[0].isupper()):
                curtrigram = prevword2 + ' ' + prevword + ' ' + word
                if(curtrigram in trigrams):
                    trigrams[curtrigram] = trigrams[curtrigram] +1
                else:
                    trigrams[curtrigram] = 1
                print(curpage,curtrigram,trigrams[curtrigram])
        prevword2 = prevword
        prevword = word
        
    #print(l)


f.close()


ch48pa02se001	  i have  now deduced from Trajan to Constantine , from Constantine to Heraclius , the regular series of the Roman emperors ; and faithfully exposed the prosperous and adverse fortunes of their reigns . 
 Trajan 1
 Constantine 1
 Constantine 2
 Heraclius 1
 Roman 1

ch48pa02se002	 five centuries of the decline and fall of the empire have already elapsed ; but a period of more than eight hundred years still separates me from the term of my labours , the taking of Constantinople by the Turks . 
 Constantinople 1
 Turks 1

ch48pa02se003	 should I persevere in the same course , should I observe the same measure , a prolix and slender thread would be spun through many a volume , nor would the patient reader find an adequate reward of instruction or amusement . 

ch48pa02se004	 at every step , as we sink deeper in the decline and fall of the Eastern empire , the annals of each succeeding reign would impose a more ungrateful and melancholy task . 
 Eastern 1

ch48pa02se005	 the

ch48pa13se006	 the writers , whose awkward satire is praise , describe him as an itinerant pedlar , who drove an ass with some paltry merchandise to the country fairs ; and foolishly relate that he met on the road some Jewish fortune-tellers , who promised him the Roman empire on condition that he should abolish the worship of idols . 
v08p235 Jewish 1
v08p235 Roman 16

ch48pa13se007	 a more probable account relates the migration of his father from Asia Minor to Thrace , where he exercised the lucrative trade of a grazier ; and he must have acquired considerable wealth , since the first introduction of his son was procured by a supply of five hundred sheep to the Imperial camp . 
v08p235 Asia 3
v08p235 Minor 1
v08p235 Asia Minor 1
v08p235 Thrace 2
v08p235 Imperial 7

ch48pa13se008	 his first service was in the guards of Justinian , where he soon attracted the notice , and by degrees the jealousy , of the tyrant . 
v08p235 Justinian 18

ch48pa13se009	 his valour and dexterity were consp

ch48pa23se005	  folly ; and it was impossible , without forfeiting the public esteem , to acquire or preserve the favour of the emperor . 

ch48pa23se006	 the millions of gold and silver which had been accumulated for the service of the state were lavished on the vilest of men , who flattered his passions and shared his pleasures ; and , in a reign of thirteen years , the richest of sovereigns was compelled to strip the palace and the churches of their precious furniture . 

ch48pa23se007	 like Nero , he delighted in the amusements of the theatre , and sighed to be surpassed in the accomplishments in which he should have blushed to excel . 
v08p254 Nero 3

ch48pa23se008	 yet the studies of Nero in music and poetry betrayed some symptoms of a liberal taste ; the more ignoble arts of the son of Theophilus were confined to the chariot-race of the hippodrome . 
v08p254 Nero 4
v08p254 Theophilus 10

ch48pa23se009	 the four factions which had agitated the peace , still amused the idleness , 

ch48pa32se005	 Basil was the acknowledged sovereign of Constantinople and the provinces of Europe ; but Asia was oppressed by two veteran generals , Phocas and 
v08p273 Basil 20
v08p273 Constantinople 37
v08p273 Europe 5
v08p273 Asia 5
v08p273 Phocas 2

ch48pa32se005	  Sclerus , who , alternately friends and enemies , subjects and rebels , maintained their independence , and laboured to emulate the example of successful usurpation .  
v08p274 Sclerus 1

ch48pa32se006	 against these domestic enemies , the son of Romanus first drew his sword , and they trembled in the presence of a lawful and high-spirited prince . 
v08p274 Romanus 11

ch48pa32se007	 the first , in the front of battle , was thrown from his horse , by the stroke of poison or an arrow ; the second , who had been twice loaded with chains , and twice invested with the purple , was desirous of ending in peace the small remainder of his days . 

ch48pa32se008	 as the aged suppliant approached the throne , with dim eyes and fal

v08p290 Roman 35

ch48pa38se011	 severe to himself , indulgent to others , chaste , frugal , abstemious , the philosophic Marcus would not have disdained the artless virtues of his successor , derived from his heart , and not borrowed from the schools . 
v08p290 Marcus 1

ch48pa38se012	 he despised and moderated the stately magnificence of the Byzantine court , so oppressive to the people , so contemptible to the eye of reason . 
v08p290 Byzantine 15

ch48pa38se013	 under such a prince , innocence had nothing to fear , and merit had everything to hope ; and , without assuming the tyrannic office of a censor , he introduced a gradual , though visible , reformation in the public and private manners of Constantinople . 
v08p290 Constantinople 46

ch48pa38se014	 the only defect of this accomplished character was the frailty of noble minds , the love of arms and military glory . 

ch48pa38se015	 yet the frequent expeditions of John the Handsome may be justified , at least in their principle

v08p303 Marius 1
v08p303 Tiberius 4
v08p303 Andronicus 16

ch48pa45se007	 his memory was stored with a black list of the enemies and rivals , who had traduced his merit , opposed his greatness , or insulted his misfortunes ; 

ch48pa45se007	  and the only comfort of his exile was the sacred hope and promise of revenge . 

ch48pa45se008	 the necessary extinction of the young emperor and his mother imposed the fatal obligation of extirpating the friends who hated and might punish the assassin ; and the repetition of murder rendered him less willing , and less able , to forgive . 

ch48pa45se009	 an horrid narrative of the victims whom he sacrificed by poison or the sword , by the sea or the flames , would be less expressive of his cruelty than the appellation of the Halcyon-days , which was applied to a rare and bloodless week of repose . 
v08p304 Halcyon-days 1

ch48pa45se010	 the tyrant strove to transfer , on the laws and the judges , some portion of his guilt ; but the mask was falle

In [25]:
workl = '	  of Egypt '

args = workl.split()
args[0]

'of'